# Clasificación de deportes a partir de vídeos - RNN

## 1. Introducción

- Objetivo: Clasificación de vídeos en 5 deportes distintos usando CNN+LSTM
- Dataset: UCF50
- Arquitectura: CNN para características espaciales + LSTM para temporales

## 2. Importación de librerías

In [ ]:
import os
from keras.utils import to_categorical
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense, TimeDistributed, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import cv2


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Andreu Picornell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Andreu Picornell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py"

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Andreu Picornell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Andreu Picornell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py"

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

## 3. Carga y preprocesamiento de datos

In [ ]:
IMG_SIZE = 224
NUM_FRAMES = 20

def extract_frames(video_path, num_frames=NUM_FRAMES):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = max(1, total_frames // num_frames)

    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * step)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frame = frame / 255.0
        frames.append(frame)
    cap.release()

    while len(frames) < num_frames:
        frames.append(np.zeros((IMG_SIZE, IMG_SIZE, 3)))
    return np.array(frames)


## 4. Construcción del modelo

In [ ]:
def build_model(num_classes=5):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    cnn = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))

    model = Sequential([
        TimeDistributed(cnn, input_shape=(NUM_FRAMES, IMG_SIZE, IMG_SIZE, 3)),
        LSTM(64),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

## 5. Entrenamiento del modelo

In [ ]:
def load_data(data_dir):
    X, y, labels = [], [], sorted(os.listdir(data_dir))
    label_map = {label: idx for idx, label in enumerate(labels)}
    
    for label in labels:
        class_dir = os.path.join(data_dir, label)
        for video_file in os.listdir(class_dir):
            video_path = os.path.join(class_dir, video_file)
            frames = extract_frames(video_path)
            X.append(frames)
            y.append(label_map[label])
    
    return np.array(X), to_categorical(y), label_map

In [ ]:
X_train, y_train, label_map = load_data("dataset/train")
X_test, y_test, _ = load_data("dataset/test")

model = build_model(num_classes=5)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=4, validation_split=0.2)

## 6. Evaluación

In [ ]:
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Etiquetas de clase
labels = list(label_map.keys())

# Imprimir métricas
print("📊 Reporte de Clasificación:\n")
print(classification_report(y_true, y_pred, target_names=labels))

# Matriz de confusión
cm = confusion_matrix(y_true, y_pred)

# Visualizar matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels, cmap="Blues")
plt.xlabel('Predicción')
plt.ylabel('Etiqueta real')
plt.title('🔍 Matriz de Confusión')
plt.show()

## 7.  Función de Inferencia

In [ ]:
def predict_video(video_path, model, label_map):
    frames = extract_frames(video_path)
    frames = np.expand_dims(frames, axis=0)  # Batch dimension
    prediction = model.predict(frames)
    predicted_class = np.argmax(prediction)
    inv_label_map = {v: k for k, v in label_map.items()}
    return inv_label_map[predicted_class]